# Test Notebook

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [2]:
# In a Jupyter notebook or IPython environment, run this in the first cell
%load_ext autoreload
%autoreload 2

In [12]:
%reload_ext autoreload

In [26]:
from models import SiameseResNetUNet
model = SiameseResNetUNet(
    in_channels=3,
    out_channels=5,
    backbone_name="resnet18",
    pretrained=True,
    freeze_backbone=True,
    mode="conc")

In [5]:
from datasets import xDB_Siamese_Dataset
from training.augmentations import get_val_augmentation_pipeline

data = xDB_Siamese_Dataset(origin_dir="../data/data_samples/xDB/tier3", 
                           transform=get_val_augmentation_pipeline()
                           )

Loaded 3 train labels.


In [6]:
from torch.utils.data import DataLoader
train_dl = DataLoader(
    dataset=data,
    batch_size=1,
    shuffle=True
)

In [ ]:
from training import train
import torch.nn as nn
import torch 
from metrics import iou_score, balanced_accuracy, f1_score
from losses import DiceLoss
import torch.optim as optim

mode = "multiclass"
criterion = DiceLoss(mode=mode)
metrics = [f1_score, iou_score,  balanced_accuracy]
class_weights = [1, 10, 20, 30, 40]

optimizer = optim.AdamW(params=filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-6)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 11, 17, 25, 33, 47, 50, 60, 70, 90, 110, 130, 150, 170, 180, 190], gamma=0.5)

train(
    model = model,
    train_dl = train_dl,
    valid_dl = train_dl,
    loss_fn = criterion,
    optimizer = optimizer, 
    scheduler = scheduler, 
    metrics = metrics,
    nb_epochs = 50,
    experiment_name = "Test",
    log_dir="../runs",
    model_dir="../models",
    resume_path=None,
    early_stopping_params = None,
    image_key = "post_image",
    mask_key = "post_mask",
    num_classes = 5, 
    verbose= False,  # Adding verbose flag
    checkpoint_interval = 10,  # Add checkpoint interval parameter
    debug = False,  # Add debug flag for memory logging, 
    training_log_interval = 1, 
    is_mixed_precision=True,
    reduction= "weighted",
    class_weights = class_weights,
    siamese = True
)

INFO:root:Experiment logs are recorded at ../runs\Test_20241205-195536
INFO:root:Epoch 1
INFO:root:--------------------
INFO:root:Epoch 1 - Validation Phase
INFO:root:--------------------
Validation Epoch 1: 100%|██████████| 3/3 [00:00<00:00,  6.66batch/s, Loss=0.879]
INFO:root:Epoch 2
INFO:root:--------------------
INFO:root:Epoch 2 - Validation Phase
INFO:root:--------------------
Validation Epoch 2: 100%|██████████| 3/3 [00:00<00:00,  7.10batch/s, Loss=0.878]
INFO:root:Epoch 3
INFO:root:--------------------
INFO:root:Epoch 3 - Validation Phase
INFO:root:--------------------
Validation Epoch 3: 100%|██████████| 3/3 [00:00<00:00,  5.67batch/s, Loss=0.878]
INFO:root:Epoch 4
INFO:root:--------------------
INFO:root:Epoch 4 - Validation Phase
INFO:root:--------------------
Validation Epoch 4: 100%|██████████| 3/3 [00:00<00:00,  6.01batch/s, Loss=0.876]
INFO:root:Epoch 5
INFO:root:--------------------
INFO:root:Epoch 5 - Validation Phase
INFO:root:--------------------
Validation Epoch 5: 

In [21]:
t = torch.randn(size=(1,512,512))
binary_mask = torch.where(t > 0 )
binary_mask.shape()

AttributeError: 'tuple' object has no attribute 'shape'

### Test New Models 

In [3]:
from models import Segformer, Unet, ResNet_Unet
model_name = "nvidia/segformer-b0-finetuned-ade-512-512"
label2id = {"building": 1, "background": 0 } #{"cloud": 1, "no_cloud": 0 }
id2label = {v: k for k,v in label2id.items()}
num_labels = 2
freeze_encoder = True

model = Segformer(model_name=model_name,
                  label2id=label2id,
                  num_labels=num_labels,
                  freeze_encoder=freeze_encoder
                  )

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([2, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Encoder weights have been frozen.


In [50]:
from datasets import  Puerto_Rico_Building_Dataset
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from training.augmentations import get_val_augmentation_pipeline

# Define Albumentations transforms with normalization
transform = get_val_augmentation_pipeline(image_size=(512, 512), max_pixel_value=1, mean=(0,0,0), std=(1,1,1))

### Test Dataset 

In [51]:
from models import AutoEncoder
from training.augmentations import get_val_augmentation_pipeline
import torch.nn as nn

cloud_filter_params = {"model_class": AutoEncoder(num_input_channel=3,base_channel_size=64), 
                        "device": "cuda", 
                        "file_path": "../models/AutoEncoder_Cloud_Detector_0.001297.pth",
                        "threshold": 0.001297, 
                        "loss": nn.MSELoss,
                        "batch_size": 32
                        }

In [72]:
data_puerto = Puerto_Rico_Building_Dataset(
    base_dir="../data/Puerto_Rico_dataset/tiff_tiles",
    pre_disaster_dir="Pre_Event_Grids_In_TIFF",
    post_disaster_dir="Post_Event_Grids_In_TIFF",
    mask_dir="Post_Event_Grids_In_TIFF_mask",
    transform=transform,
    extension="tif",
    cloud_filter_params=cloud_filter_params,
    preprocessing_mode="online",
    filtered_list_path=None
    )

<class 'models.Auto_Encoder.AutoEncoder'> Model loaded from ../models/AutoEncoder_Cloud_Detector_0.001297.pth
A AutoEncoder model has been loaded from ../models/AutoEncoder_Cloud_Detector_0.001297.pth on cuda.


In [100]:
from models import ResNet_UNET

model = ResNet_UNET(
        in_channels=3,
        out_channels=2,
        backbone_name="resnet18",
        pretrained=True,
        freeze_backbone=True,
    )
model = model.load(file_path="../models/xDB_ResNet18_Unet_20241130-201241_best_model.pth").to("cuda")

ResNet_UNET Model loaded from ../models/xDB_ResNet18_Unet_20241130-201241_best_model.pth


/home/onyxia/work/damaged-building-detector/src/models/ResNet_Unet.py:157: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(file_path))


In [101]:
data_dl = DataLoader(dataset=data_puerto, batch_size=16, shuffle=True)

In [102]:
inputs = next(iter(data_dl))
outputs = model.predict(inputs["pre_image"].to("cuda"))

In [ ]:
from utils import display_semantic_predictions_batch

display_semantic_predictions_batch(images=inputs["pre_image"], 
                                    mask_predictions=outputs,
                                    mask_labels=inputs["mask"], 
                                    normalized=None, 
                                    folder_path=None
                                    )

In [105]:
from models import Maskrcnn
# Initialize the Mask R-CNN model
from models import Maskrcnn
maskrcnn = Maskrcnn(num_classes=2, hidden_layer_dim=256, pretrained=False)
maskrcnn.load("../models/xDB_ResNet50_MaskRCNN_checkpoint.pth")

Model loaded from ../models/xDB_ResNet50_MaskRCNN_checkpoint.pth


In [ ]:
from utils import display_semantic_predictions_batch
import numpy as np
import torch
maskrcnn.eval()
with torch.no_grad():
    images = inputs["pre_image"].to("cuda")
    mask_predictions = maskrcnn.predict_sem_seg(images = images)

In [ ]:
display_semantic_predictions_batch(images=inputs["pre_image"], 
                                    mask_predictions=mask_predictions,
                                    mask_labels=inputs["mask"], 
                                    normalized=None, 
                                    folder_path=None
                                    )

In [ ]:
from utils import display_instance_predictions_batch
maskrcnn.eval()
with torch.no_grad():
    images = inputs["pre_image"].to("cuda")
    instances_predictions = maskrcnn.predict(images = images)
    
display_instance_predictions_batch(images, mask_predictions, score_threshold=0.6, max_images=len(images), display=["mask","boxes"])